In [83]:
'''
This is to scrape verizonwireless.com's iphone reviews.
There are 179 reviews for iphone X and 73 reviews for galaxy s9.

written by Soi Kang
3/30/2019
'''

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
from bs4 import BeautifulSoup

driver = webdriver.Chrome()
driver.get("https://www.verizonwireless.com/smartphones/apple-iphone-x/?sku=sku2680533")

# need to navigate through reviews.
# problem for this website is that the url doesn't change no matter what I navigate through
# but selenium can do it

#this is to get to the reviews from the original url
review_button_path = '//*[@id="reviewsLink"]'
wait = WebDriverWait(driver, 5)
review_button = wait.until(EC.visibility_of_element_located((By.XPATH, review_button_path)))
driver.execute_script("arguments[0].click();", review_button)

In [84]:
# get the reviews and rating for the first page
reviews = []
ratings = []
review_rating = []

review_list = driver.find_elements_by_css_selector('.NHaasDS55Rg.fontSize_12.padBottom15.wrapText')
for el in review_list:
    #print(el.find_element_by_tag_name('span').text)
    reviews.append(el.find_element_by_tag_name('span').text)
    
rating_list = driver.find_elements_by_css_selector('.displayInlineBlock.positionRelative.fontSize_14')

for i in range(len(rating_list)) :
    print (i)
    # we should get rid of the first 7 elements since it is different rating values; not associated with the reviews.
    # starting index at 7, the stars are associated with the reviews
    if(i>6) :
        element = rating_list[i].find_element_by_class_name("a11y-hidden").text
        ratings.append(element) #this will have 17 elements in the list.
    else :
        pass #do nothing
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [85]:
# to get the rest of the review pages
from selenium.common import exceptions
import time

j=0
while(True):
    try :
        next_review_path = '//*[@id="reviews"]/div/div[3]/div/ul/li[6]/a/span[1]' 
        review_next_button = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH, next_review_path)))
        driver.execute_script("arguments[0].click();", review_next_button)
        time.sleep(4)
  
        review_list = driver.find_elements_by_css_selector('.NHaasDS55Rg.fontSize_12.padBottom15.wrapText')
        for el in review_list:
            reviews.append(el.find_element_by_tag_name('span').text)
    
        rating_list = driver.find_elements_by_css_selector('.displayInlineBlock.positionRelative.fontSize_14')
        for i in range(len(rating_list)) :
        # we should get rid of the first 7 elements since it is different rating values; not associated with the reviews.
        # starting index at 7, the stars are associated with the reviews
            if(i>6) :
                element = rating_list[i].find_element_by_class_name("a11y-hidden").text
                ratings.append(element) #this will have 17 elements in the list.
            else :
                pass #do nothing
    
        time.sleep(4)
        print(j)
        j=j+1
    
    except exceptions.StaleElementReferenceException as e:
        raise e
    
    except exceptions.TimeoutException :
        break
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [87]:
len(reviews) #was time.sleep(3) there were 160 elements in the list


170

In [88]:
len(ratings)

170

In [97]:
# combine the two lists as a dataframe
import pandas as pd
import re

numbers = []
for rating in ratings :
    
    numbers.append(re.search(r'\d+', rating).group())

df = pd.DataFrame({"review" : reviews, "rating" : numbers})
df['source'] = "vw"
df['phone'] = "iphone"
    
    
#reorder columns
cols = ['source','phone','review','rating']
final = df.reindex(columns=cols)        

In [98]:
final.to_csv('vw_iphone.csv', index=False)